In [7]:
# ============================================================
# AetherVision Omega - HyperCity Core GeoAI Engine (Branch 1)
# ============================================================

# -----------------------------
# STEP 0: Imports and folders
# -----------------------------
import os
import json
import math
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster

BASE_DIR = "AetherVision_Omega"
DASH_DIR = os.path.join(BASE_DIR, "dashboard")
DATA_DIR = os.path.join(BASE_DIR, "data")

os.makedirs(DASH_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# -----------------------------
# Helper: load or create India boundary
# -----------------------------
india_boundary_path = os.path.join(DATA_DIR, "india_boundary.geojson")

def get_india_geojson():
    if os.path.exists(india_boundary_path):
        with open(india_boundary_path, "r", encoding="utf-8") as f:
            return json.load(f)
    # Fallback simple bounding polygon (approximate)
    poly = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {"name": "India Approx"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [68.0, 6.0],
                    [97.5, 6.0],
                    [97.5, 37.0],
                    [68.0, 37.0],
                    [68.0, 6.0]
                ]]
            }
        }]
    }
    with open(india_boundary_path, "w", encoding="utf-8") as f:
        json.dump(poly, f)
    return poly

india_geo = get_india_geojson()

# Center coordinates
CENTER_LAT = 22.5937
CENTER_LON = 78.9629

# ============================================================
# STEP 1: Digital Twin Base Map
# ============================================================
digital_twin = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#d0e6f7",
        "color": "#0055A4",
        "weight": 2,
        "fillOpacity": 0.25,
    }
).add_to(digital_twin)

base_out = os.path.join(DASH_DIR, "digital_twin_base.html")
digital_twin.save(base_out)
print("Saved:", base_out)

# ============================================================
# STEP 2: Environmental Intelligence Layer
# ============================================================

# 2.1 AQI data (sample)
aqi = pd.DataFrame({
    "city": [
        "Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad",
        "Bengaluru", "Kanpur", "Lucknow", "Ahmedabad", "Bhubaneswar"
    ],
    "lat": [28.7041, 19.0760, 22.5726, 13.0827, 17.3850,
            12.9716, 26.4499, 26.8467, 23.0225, 20.2961],
    "lon": [77.1025, 72.8777, 88.3639, 80.2707, 78.4867,
            77.5946, 80.3319, 80.9462, 72.5714, 85.8245],
    "aqi": [420, 260, 190, 180, 210, 170, 390, 360, 230, 160]
})

# 2.2 Industrial zones (sample)
industrial = pd.DataFrame({
    "zone": [
        "Vapi Industrial Belt", "Ankleshwar Chemical Zone",
        "Vizag Corridor", "Taloja MIDC", "Noida Industrial Zone",
        "Koraput Industrial Cluster"
    ],
    "lat": [20.3895, 21.6252, 17.6868, 19.0875, 28.5355, 18.8110],
    "lon": [72.9106, 72.9960, 83.2185, 73.1031, 77.3910, 82.7112]
})

# 2.3 River quality points (sample)
rivers = pd.DataFrame({
    "river": ["Ganga", "Yamuna", "Godavari", "Mahanadi"],
    "lat": [25.4358, 27.1767, 17.0050, 20.2670],
    "lon": [81.8463, 78.0081, 81.7890, 85.8430],
    "pollution_index": [0.85, 0.78, 0.52, 0.60]
})

env_map = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

# AQI heatmap
HeatMap(
    aqi[["lat", "lon", "aqi"]].values.tolist(),
    name="AQI Heatmap",
    radius=40,
    blur=25,
    min_opacity=0.4,
).add_to(env_map)

# Industrial markers
cluster = MarkerCluster(name="Industrial Zones").add_to(env_map)
for _, row in industrial.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"Industrial Zone: {row['zone']}",
        icon=folium.Icon(color="red", icon="industry", prefix="fa")
    ).add_to(cluster)

# River quality markers
for _, row in rivers.iterrows():
    score = row["pollution_index"]
    color = "green" if score < 0.4 else "orange" if score < 0.7 else "red"
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        popup=f"{row['river']} pollution index: {score:.2f}",
        color=color,
        fill=True,
        fill_opacity=0.8
    ).add_to(env_map)

# India boundary overlay
folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#00000000",
        "color": "#0055A4",
        "weight": 1.5,
        "fillOpacity": 0.0,
    }
).add_to(env_map)

env_out = os.path.join(DASH_DIR, "india_env_layers.html")
env_map.save(env_out)
print("Saved:", env_out)

# ============================================================
# STEP 3: Geological Hazard Layer
# ============================================================

# 3.1 Earthquake hotspots (sample)
eq = pd.DataFrame({
    "zone": [
        "Himalayan Belt", "North-East India",
        "Kutch Region", "Andaman-Nicobar"
    ],
    "lat": [30.0, 26.0, 23.5, 11.6],
    "lon": [79.0, 94.0, 70.0, 92.7],
    "seismic_intensity": [0.95, 0.88, 0.76, 0.82]
})

# 3.2 Landslide hotspots (sample)
landslide = pd.DataFrame({
    "name": ["Uttarakhand Hills", "Darjeeling Hills", "Nilgiri Hills"],
    "lat": [30.3, 27.0, 11.4],
    "lon": [79.6, 88.3, 76.7],
    "risk": [0.92, 0.80, 0.74]
})

# 3.3 Fault line simplified segments (sample)
fault_segments = [
    [(72.0, 23.0), (78.0, 30.0)],  # NW India
    [(88.0, 26.0), (95.0, 27.5)],  # NE India
    [(76.0, 10.0), (80.0, 14.0)],  # South fault
]

geo_map = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

# Earthquake heatmap
HeatMap(
    eq[["lat", "lon", "seismic_intensity"]].values.tolist(),
    radius=40,
    blur=25,
    name="Seismic Hotspots",
    min_opacity=0.4
).add_to(geo_map)

# Landslide markers
for _, row in landslide.iterrows():
    r = row["risk"]
    color = "red" if r > 0.85 else "orange"
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        popup=f"Landslide area: {row['name']} (risk={r:.2f})",
        color=color,
        fill=True,
        fill_opacity=0.9
    ).add_to(geo_map)

# Fault lines
for seg in fault_segments:
    folium.PolyLine(
        locations=[(y, x) for x, y in seg],  # (lon, lat) -> (lat, lon)
        popup="Major Fault Line",
        color="#8B0000",
        weight=3,
        dash_array="5, 7"
    ).add_to(geo_map)

folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#00000000",
        "color": "#0055A4",
        "weight": 1.5,
        "fillOpacity": 0.0,
    }
).add_to(geo_map)

geo_out = os.path.join(DASH_DIR, "geology_layers.html")
geo_map.save(geo_out)
print("Saved:", geo_out)

# ============================================================
# STEP 4: Climate and Urban Stress Layer + Smart Solutions
# ============================================================

# 4.1 Urban heat islands (sample)
uhi = pd.DataFrame({
    "city": ["Delhi", "Ahmedabad", "Nagpur", "Hyderabad", "Bengaluru"],
    "lat": [28.7041, 23.0225, 21.1458, 17.3850, 12.9716],
    "lon": [77.1025, 72.5714, 79.0882, 78.4867, 77.5946],
    "heat_index": [0.93, 0.82, 0.78, 0.76, 0.70]
})

# 4.2 Groundwater stress (sample)
gw = pd.DataFrame({
    "area": ["North Gujarat", "Punjab Plain", "Marathwada", "Bundelkhand"],
    "lat": [23.5, 30.5, 19.3, 25.0],
    "lon": [72.0, 75.0, 76.1, 80.0],
    "stress": [0.88, 0.91, 0.85, 0.83]
})

# 4.3 Smart solution proposals (micro-forests, recharge)
solutions = pd.DataFrame({
    "type": ["Micro-Forest", "Recharge Park", "Cool Roof Pilot", "Green Corridor"],
    "lat": [28.6, 23.1, 19.1, 17.4],
    "lon": [77.2, 72.7, 72.9, 78.5],
    "description": [
        "Dense native tree plantation for UHI reduction",
        "Groundwater recharge and wetland restoration zone",
        "High-albedo cool roof testbed on public buildings",
        "EV-priority, tree-lined transport corridor"
    ]
})

climate_map = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

# UHI heatmap
HeatMap(
    uhi[["lat", "lon", "heat_index"]].values.tolist(),
    radius=45,
    blur=30,
    name="Urban Heat Island Intensity",
    min_opacity=0.4
).add_to(climate_map)

# Groundwater stress markers
for _, row in gw.iterrows():
    s = row["stress"]
    color = "red" if s > 0.9 else "orange"
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=9,
        popup=f"Groundwater stress: {row['area']} (index={s:.2f})",
        color=color,
        fill=True,
        fill_opacity=0.9
    ).add_to(climate_map)

# Smart city solutions
for _, row in solutions.iterrows():
    icon_color = "green"
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"{row['type']}: {row['description']}",
        icon=folium.Icon(color=icon_color, icon="leaf", prefix="fa")
    ).add_to(climate_map)

folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#00000000",
        "color": "#0055A4",
        "weight": 1.5,
        "fillOpacity": 0.0,
    }
).add_to(climate_map)

smartcity_out = os.path.join(DASH_DIR, "omega_smartcity.html")
climate_map.save(smartcity_out)
print("Saved:", smartcity_out)

# ============================================================
# STEP 5: AI Fusion Risk Engine
# ============================================================

# Build a unified table for major cities with multi-hazard scores
cities = pd.DataFrame({
    "city": ["Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad", "Ahmedabad", "Bengaluru"],
    "lat": [28.7041, 19.0760, 22.5726, 13.0827, 17.3850, 23.0225, 12.9716],
    "lon": [77.1025, 72.8777, 88.3639, 80.2707, 78.4867, 72.5714, 77.5946]
})

# Attach environmental scores by merging with previous data
cities = cities.merge(
    aqi[["city", "aqi"]],
    on="city",
    how="left"
).rename(columns={"aqi": "air_raw"})

# Normalize helper
def min_max(series):
    if series.max() == series.min():
        return series * 0
    return (series - series.min()) / (series.max() - series.min())

# Air pollution normalized
cities["air_norm"] = min_max(cities["air_raw"].fillna(cities["air_raw"].mean()))

# Attach UHI
cities = cities.merge(
    uhi[["city", "heat_index"]],
    on="city",
    how="left"
)
cities["heat_norm"] = min_max(cities["heat_index"].fillna(0.5))

# Seismic approximate by distance to nearest eq hotspot
def nearest_eq_intensity(lat, lon):
    dists = []
    for _, row in eq.iterrows():
        d = math.sqrt((lat - row["lat"])**2 + (lon - row["lon"])**2)
        dists.append((d, row["seismic_intensity"]))
    dists.sort(key=lambda x: x[0])
    return dists[0][1] if dists else 0.5

cities["eq_norm"] = cities.apply(lambda r: nearest_eq_intensity(r["lat"], r["lon"]), axis=1)

# Approx groundwater stress by nearest gw point
def nearest_gw(lat, lon):
    dists = []
    for _, row in gw.iterrows():
        d = math.sqrt((lat - row["lat"])**2 + (lon - row["lon"])**2)
        dists.append((d, row["stress"]))
    dists.sort(key=lambda x: x[0])
    return dists[0][1] if dists else 0.5

cities["gw_norm"] = cities.apply(lambda r: nearest_gw(r["lat"], r["lon"]), axis=1)

# Industrial influence by distance to nearest zone
def nearest_industrial(lat, lon):
    dists = []
    for _, row in industrial.iterrows():
        d = math.sqrt((lat - row["lat"])**2 + (lon - row["lon"])**2)
        dists.append(d)
    if not dists:
        return 0.3
    dmin = min(dists)
    # closer -> higher risk
    val = max(0.0, 1.0 - dmin / 15.0)
    return min(val, 1.0)

cities["industry_norm"] = cities.apply(lambda r: nearest_industrial(r["lat"], r["lon"]), axis=1)

# Landslide influence (very approximate, only relevant for some cities)
def nearest_landslide(lat, lon):
    dists = []
    for _, row in landslide.iterrows():
        d = math.sqrt((lat - row["lat"])**2 + (lon - row["lon"])**2)
        dists.append((d, row["risk"]))
    if not dists:
        return 0.2
    dists.sort(key=lambda x: x[0])
    return dists[0][1] if dists[0][0] < 5 else 0.2

cities["landslide_norm"] = cities.apply(lambda r: nearest_landslide(r["lat"], r["lon"]), axis=1)

# Fusion risk formula
w_eq = 0.30
w_ls = 0.20
w_air = 0.15
w_ind = 0.15
w_heat = 0.10
w_gw = 0.10

cities["fusion_risk"] = (
    w_eq * cities["eq_norm"] +
    w_ls * cities["landslide_norm"] +
    w_air * cities["air_norm"] +
    w_ind * cities["industry_norm"] +
    w_heat * cities["heat_norm"] +
    w_gw * cities["gw_norm"]
)

fusion_map = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

# Fusion heatmap
HeatMap(
    cities[["lat", "lon", "fusion_risk"]].values.tolist(),
    radius=50,
    blur=35,
    name="Fusion Risk Index",
    min_opacity=0.4
).add_to(fusion_map)

# City markers with breakdown popup
for _, row in cities.iterrows():
    popup = folium.Popup(
        html=(
            f"<b>{row['city']}</b><br>"
            f"Fusion Risk: {row['fusion_risk']:.2f}<br>"
            f"Air: {row['air_norm']:.2f}<br>"
            f"Heat: {row['heat_norm']:.2f}<br>"
            f"Seismic: {row['eq_norm']:.2f}<br>"
            f"Landslide: {row['landslide_norm']:.2f}<br>"
            f"Industry: {row['industry_norm']:.2f}<br>"
            f"Groundwater: {row['gw_norm']:.2f}"
        ),
        max_width=250
    )
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=7,
        popup=popup,
        color="black",
        fill=True,
        fill_opacity=0.9
    ).add_to(fusion_map)

folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#00000000",
        "color": "#0055A4",
        "weight": 1.5,
        "fillOpacity": 0.0,
    }
).add_to(fusion_map)

fusion_out = os.path.join(DASH_DIR, "fusion_risk_map.html")
fusion_map.save(fusion_out)
print("Saved:", fusion_out)

# ============================================================
# STEP 6: Integrated Dashboard
# ============================================================

dashboard = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=5,
    tiles="cartodbpositron"
)

# Feature groups
fg_aqi = folium.FeatureGroup(name="AQI Heatmap").add_to(dashboard)
fg_ind = folium.FeatureGroup(name="Industrial Zones").add_to(dashboard)
fg_river = folium.FeatureGroup(name="River Pollution").add_to(dashboard)
fg_eq = folium.FeatureGroup(name="Seismic Hotspots").add_to(dashboard)
fg_ls = folium.FeatureGroup(name="Landslide Zones").add_to(dashboard)
fg_fault = folium.FeatureGroup(name="Fault Lines").add_to(dashboard)
fg_uhi = folium.FeatureGroup(name="Urban Heat Islands").add_to(dashboard)
fg_gw = folium.FeatureGroup(name="Groundwater Stress").add_to(dashboard)
fg_sol = folium.FeatureGroup(name="Smart City Solutions").add_to(dashboard)
fg_fusion = folium.FeatureGroup(name="Fusion Risk Index").add_to(dashboard)

# AQI heatmap
HeatMap(
    aqi[["lat", "lon", "aqi"]].values.tolist(),
    radius=40,
    blur=25,
    min_opacity=0.4
).add_to(fg_aqi)

# Industrial markers
for _, row in industrial.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=row["zone"],
        icon=folium.Icon(color="red", icon="industry", prefix="fa")
    ).add_to(fg_ind)

# River pollution
for _, row in rivers.iterrows():
    score = row["pollution_index"]
    color = "green" if score < 0.4 else "orange" if score < 0.7 else "red"
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        popup=f"{row['river']} pollution index: {score:.2f}",
        color=color,
        fill=True,
        fill_opacity=0.8
    ).add_to(fg_river)

# Seismic
HeatMap(
    eq[["lat", "lon", "seismic_intensity"]].values.tolist(),
    radius=40,
    blur=25,
    min_opacity=0.4
).add_to(fg_eq)

# Landslides
for _, row in landslide.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        popup=row["name"],
        color="red",
        fill=True,
        fill_opacity=0.9
    ).add_to(fg_ls)

# Fault lines
for seg in fault_segments:
    folium.PolyLine(
        locations=[(y, x) for x, y in seg],
        color="#8B0000",
        weight=3,
        dash_array="5,7"
    ).add_to(fg_fault)

# UHI
HeatMap(
    uhi[["lat", "lon", "heat_index"]].values.tolist(),
    radius=45,
    blur=30,
    min_opacity=0.4
).add_to(fg_uhi)

# Groundwater
for _, row in gw.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=9,
        popup=row["area"],
        color="orange",
        fill=True,
        fill_opacity=0.9
    ).add_to(fg_gw)

# Solutions
for _, row in solutions.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=f"{row['type']}: {row['description']}",
        icon=folium.Icon(color="green", icon="leaf", prefix="fa")
    ).add_to(fg_sol)

# Fusion risk
HeatMap(
    cities[["lat", "lon", "fusion_risk"]].values.tolist(),
    radius=50,
    blur=35,
    min_opacity=0.4
).add_to(fg_fusion)

# India boundary
folium.GeoJson(
    india_geo,
    name="India Boundary",
    style_function=lambda x: {
        "fillColor": "#00000000",
        "color": "#0055A4",
        "weight": 1.5,
        "fillOpacity": 0.0,
    }
).add_to(dashboard)

folium.LayerControl(collapsed=False).add_to(dashboard)

dashboard_out = os.path.join(DASH_DIR, "omega_dashboard.html")
# ---------------- Final Save ----------------
OUTPUT_DIR = "/content"
dashboard_path = os.path.join(OUTPUT_DIR, "omega_dashboard.html")
dashboard.save(dashboard_path)
print("Saved:", dashboard_path)

Saved: AetherVision_Omega/dashboard/digital_twin_base.html
Saved: AetherVision_Omega/dashboard/india_env_layers.html
Saved: AetherVision_Omega/dashboard/geology_layers.html
Saved: AetherVision_Omega/dashboard/omega_smartcity.html
Saved: AetherVision_Omega/dashboard/fusion_risk_map.html
Saved: /content/omega_dashboard.html


In [6]:
# Create a ZIP file containing ALL Branch 1 outputs
!zip -r AetherVision_Omega_Branch1_Outputs.zip AetherVision_Omega/dashboard

updating: AetherVision_Omega/dashboard/ (stored 0%)
updating: AetherVision_Omega/dashboard/india_env_layers.html (deflated 79%)
updating: AetherVision_Omega/dashboard/fusion_risk_map.html (deflated 78%)
updating: AetherVision_Omega/dashboard/geology_layers.html (deflated 77%)
updating: AetherVision_Omega/dashboard/digital_twin_base.html (deflated 68%)
updating: AetherVision_Omega/dashboard/omega_smartcity.html (deflated 77%)


Branch 1: HyperCity Core GeoAI Engine

Branch 1 is the main engine of our project. In this branch, we create the entire digital model of the city and add all important pollution, climate, and geological hazard layers. We use Python, Folium (for maps), GeoPandas (for geo-data) and Pandas (for datasets). In the first step, we create a “Digital Twin Base,” which means we build a digital version of India using a boundary GeoJSON file. This gives us a clean base map where we can later add all risk layers.

In the second step, we add environmental layers. These include air pollution (AQI heatmap), industrial emission zones, and water pollution points. Air pollution shows where cities are facing high AQI, industrial zones show areas with chemical or factory emissions, and water pollution points highlight contaminated river sections. All these layers help us understand how pollution spreads in a city.

In the third step, we add geological hazard layers like earthquake points, landslide zones, and fault lines. These hazards are important because geology has a strong impact on city safety. Earthquake data shows high seismic areas, landslide zones show unstable slopes, and fault lines show tectonic movement paths.

In the fourth step, we add climate and urban stress layers. These include Urban Heat Islands (places getting hotter), groundwater stress zones (places with water scarcity), and eco-barrier/green infrastructure zones for better climate resilience. These layers help a city prepare for climate change and manage resources better.

After all layers are added, we build the “AI Fusion Risk Engine.” This engine combines all hazards into one single Smart City Risk Score. We give weights to each layer — for example, earthquake has 30% weight, landslide has 20%, air pollution 15%, industrial zones 15%, heat 10%, and groundwater 10%. This formula calculates a final risk score for each city. This step is important because the city administration always needs a single score to take decisions quickly.

In the last step, we create the final “HyperCity Dashboard.” This dashboard includes all map layers together. The user can turn ON or OFF any layer like air pollution, industrial zones, earthquakes, landslides, heat islands, groundwater stress, or the final fusion risk map. The dashboard gives a clean, interactive, and professional view of the entire city’s risk profile. It also helps in planning, monitoring, emergency response, and building safer and smarter cities.Branch 1: HyperCity Core GeoAI Engine – Full Explanation (Simple English)

Branch 1 is the main engine of our project. In this branch, we create the entire digital model of the city and add all important pollution, climate, and geological hazard layers. We use Python, Folium (for maps), GeoPandas (for geo-data) and Pandas (for datasets). In the first step, we create a “Digital Twin Base,” which means we build a digital version of India using a boundary GeoJSON file. This gives us a clean base map where we can later add all risk layers.

In the second step, we add environmental layers. These include air pollution (AQI heatmap), industrial emission zones, and water pollution points. Air pollution shows where cities are facing high AQI, industrial zones show areas with chemical or factory emissions, and water pollution points highlight contaminated river sections. All these layers help us understand how pollution spreads in a city.

In the third step, we add geological hazard layers like earthquake points, landslide zones, and fault lines. These hazards are important because geology has a strong impact on city safety. Earthquake data shows high seismic areas, landslide zones show unstable slopes, and fault lines show tectonic movement paths.

In the fourth step, we add climate and urban stress layers. These include Urban Heat Islands (places getting hotter), groundwater stress zones (places with water scarcity), and eco-barrier/green infrastructure zones for better climate resilience. These layers help a city prepare for climate change and manage resources better.

After all layers are added, we build the “AI Fusion Risk Engine.” This engine combines all hazards into one single Smart City Risk Score. We give weights to each layer — for example, earthquake has 30% weight, landslide has 20%, air pollution 15%, industrial zones 15%, heat 10%, and groundwater 10%. This formula calculates a final risk score for each city. This step is important because the city administration always needs a single score to take decisions quickly.

In the last step, we create the final “HyperCity Dashboard.” This dashboard includes all map layers together. The user can turn ON or OFF any layer like air pollution, industrial zones, earthquakes, landslides, heat islands, groundwater stress, or the final fusion risk map. The dashboard gives a clean, interactive, and professional view of the entire city’s risk profile. It also helps in planning, monitoring, emergency response, and building safer and smarter cities.